In [37]:
import json

filename = "pandas_function_openai.json"

with open(filename, "r", encoding="utf-8") as json_file:
        # Load the JSON data from the file
        data = json.load(json_file)

In [38]:
filename = "parent_summary_dict.json"

with open(filename, "r", encoding="utf-8") as json_file:
        # Load the JSON data from the file
        parent_summary_dict = json.load(json_file)

In [27]:
import networkx as nx

pandas_graph = nx.DiGraph()

parent_names = [(data[d]['name'],{"url":data[d]['url'],"url":data[d]['url'],"type":"parent_node","node_description":parent_summary_dict[data[d]['name']]}) for d in data]
pandas_graph.add_nodes_from(parent_names)

In [28]:
for parent in data:
    parent_name = data[parent]['name']
    for sub_level in data[parent]['functions']:
        for func in sub_level['function_definitions']:
            func_name = func['function_name']
            # parent_trail = parent.split(".")[0]
            pandas_graph.add_nodes_from([(func_name,{'metadata':func,'trail':f"{parent_name}","type":"function_node"})])
            pandas_graph.add_edge(parent_name,func_name)

In [29]:
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
import os
from chromadb.utils.batch_utils import create_batches
from dotenv import load_dotenv,find_dotenv

load_dotenv(find_dotenv(),override=True)

True

In [32]:
import ast

docs = []
metadata = []

for node, attributes in pandas_graph.nodes(data=True):
    if attributes['type'] == 'function_node':
        docs.append(attributes['metadata']['function_text'])
        metadata.append(str(attributes['metadata']))

    elif attributes['type'] == 'parent_node':
        docs.append(attributes['node_description'])
        attributes['name'] = node
        metadata.append(str(attributes))

In [33]:
# import pprint
# pp = pprint.PrettyPrinter(depth=4)
# pp.pprint(attributes)

In [34]:
docs[0]

'The pandas library in Python provides a wide range of functions for reading and writing data in various formats. These functions include loading pickled objects, reading delimited files, CSV files, fixed-width formatted lines, Excel files, JSON strings, HTML tables, XML documents, and more. There are also functions for working with HDFStore, feather-format objects, parquet objects, ORC objects, SAS files, SPSS files, SQL databases, Google BigQuery, and Stata files. Additionally, there are functions for creating table schemas, normalizing JSON data, rendering DataFrames to XML documents, and exporting DataFrames to Stata dta format. These functions allow for seamless data manipulation and analysis across different data sources and formats. '

In [36]:
metadata[-1]

"{'function_name': 'pandas.test', 'full_function': 'pandas.test(extra_args=None, run_doctests=False)', 'function_text': 'Run the pandas test suite using pytest.', 'parameter_names_desc': [{'param_name': 'extra_args', 'param_type': 'list[str], default None', 'param_desc': 'Extra marks to run the tests.\\n'}, {'param_name': 'run_doctests', 'param_type': 'bool, default False', 'param_desc': 'Whether to only run the Python and Cython doctests. If you would like to run\\nboth doctests/regular tests, just append “–doctest-modules”/”–doctest-cython”\\nto extra_args.\\n'}], 'function_url': 'https://pandas.pydata.org/docs/reference/api/pandas.test.html#pandas.test'}"